# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
#from api_keys import geoapify_key
geoapify_key = "<removed + access deleted>" #"YOUR KEY HERE"

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0         faya  18.3851   42.4509     22.06        35          21   
1        1      farsund  58.0948    6.8047     13.30       100           0   
2        2  new norfolk -42.7826  147.0587     11.72        58          12   
3        3    jamestown  42.0970  -79.2353      5.77        77         100   
4        4      lanzhou  36.0564  103.7922     14.53        48          59   

   Wind Speed Country        Date  
0        2.60      SA  1666108228  
1        7.65      NO  1666108228  
2        1.34      AU  1666108230  
3        9.77      US  1666107934  
4        1.20      CN  1666108230

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
     "Lng", "Lat",
     geo=True,
     tiles="OSM",
     size="Humidity",
     scale=0.85,
     color="City")

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values

#create Vars for df filter or pd.query()
max_temp =  40
humidity = 40
cloudiness = 10
wind_speed = 20 

#create DF
city_data_df2 = city_data_df[
     (city_data_df['Max Temp'] < max_temp) &
     (city_data_df['Humidity'] <= humidity) &
     (city_data_df['Cloudiness'] <= cloudiness) &
     (city_data_df['Wind Speed'] < wind_speed)
     ]

# Drop any rows with null values
city_data_df2.dropna(axis=0, inplace=True)

# Display sample data
city_data_df2.head(14)

C:\Users\Aolda\AppData\Local\Temp\ipykernel_14764\3402813144.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df2.dropna(axis=0, inplace=True)


City_ID                City      Lat       Lng  Max Temp  Humidity  \
29        29         saint-louis  16.3333  -15.0000     38.80        24   
73        73               nalut  30.3333   10.8500     25.41        27   
96        96             zhangye  38.9342  100.4517     11.11        16   
141      141             brenham  30.1669  -96.3977     14.47        40   
174      174               avera  33.1940  -82.5271     12.53        26   
176      176          challapata -18.9000  -66.7667     14.15        30   
231      231          carrollton  32.9537  -96.8903     14.63        31   
239      239              rawson -43.3002  -65.1023     23.74        25   
243      243    russkaya polyana  49.4170   31.9201     12.87        32   
258      258  comodoro rivadavia -45.8667  -67.5000     25.92        24   
268      268             chengde  40.9725  117.9361      5.17        26   
331      331          swan river  52.1058 -101.2676      0.94        40   
436      436        lakki marwat  32.6079   70.9114     27.89        20   
441      441         yangliuqing  39.1375  116.9997      7.97        31   

     Cloudiness  Wind Speed Country        Date  
29            4        1.78      SN  1666108247  
73            0        5.11      LY  1666108275  
96            0        1.67      CN  1666108292  
141           0        3.09      US  1666108319  
174           0        4.90      US  1666108342  
176           0        5.26      BO  1666108343  
231           0        5.66      US  1666108361  
239           2       13.52      AR  1666108102  
243          10        3.97      UA  1666108380  
258           0        7.20      AR  1666108389  
268           0        0.63      CN  1666108395  
331           0        2.06      CA  1666108431  
436           0        1.53      PK  1666108492  
441           0        3.58      CN  1666108494

### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df2[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
29          saint-louis      SN  16.3333  -15.0000        24           
73                nalut      LY  30.3333   10.8500        27           
96              zhangye      CN  38.9342  100.4517        16           
141             brenham      US  30.1669  -96.3977        40           
174               avera      US  33.1940  -82.5271        26           
176          challapata      BO -18.9000  -66.7667        30           
231          carrollton      US  32.9537  -96.8903        31           
239              rawson      AR -43.3002  -65.1023        25           
243    russkaya polyana      UA  49.4170   31.9201        32           
258  comodoro rivadavia      AR -45.8667  -67.5000        24           
268             chengde      CN  40.9725  117.9361        26           
331          swan river      CA  52.1058 -101.2676        40           
436        lakki marwat      PK  32.6079   70.9114        20           
441         yangliuqing      CN  39.1375  116.9997        31           
476             chifeng      CN  42.2683  118.9636        20           
496              tikrit      IQ  34.6071   43.6782        14           
522             xuchang      CN  34.0167  113.8167        35           
532          mandalgovi      MN  45.7625  106.2708        23           
537            tongliao      CN  43.6125  122.2653        35           
542               bilma      NE  18.6853   12.9164        18           
550                pali      IN  25.7667   73.3333        26           
560             changli      CN  39.7042  119.1506        33

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel

categories = "accommodation.hotel"
radius = 10000
params = { "categories":categories,
           "apiKey":geoapify_key
           }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row.Lat
    longitude = row.Lng
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(14)

Starting hotel search
saint-louis - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
brenham - nearest hotel: Ant Street Inn
avera - nearest hotel: No hotel found
challapata - nearest hotel: Residencial Marian
carrollton - nearest hotel: Days Inn North Dallas/Farmers Branch
rawson - nearest hotel: Hotel Deportivo
russkaya polyana - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
chengde - nearest hotel: 紫御国际假日酒店
swan river - nearest hotel: Swan River Super 8
lakki marwat - nearest hotel: No hotel found
yangliuqing - nearest hotel: IU Inn
chifeng - nearest hotel: 心连心酒店
tikrit - nearest hotel: فندق بلازا
xuchang - nearest hotel: first hotel
mandalgovi - nearest hotel: Б
tongliao - nearest hotel: No hotel found
bilma - nearest hotel: No hotel found
pali - nearest hotel: Hotel Pawan International
changli - nearest hotel: 客房


City Country      Lat       Lng  Humidity  \
29          saint-louis      SN  16.3333  -15.0000        24   
73                nalut      LY  30.3333   10.8500        27   
96              zhangye      CN  38.9342  100.4517        16   
141             brenham      US  30.1669  -96.3977        40   
174               avera      US  33.1940  -82.5271        26   
176          challapata      BO -18.9000  -66.7667        30   
231          carrollton      US  32.9537  -96.8903        31   
239              rawson      AR -43.3002  -65.1023        25   
243    russkaya polyana      UA  49.4170   31.9201        32   
258  comodoro rivadavia      AR -45.8667  -67.5000        24   
268             chengde      CN  40.9725  117.9361        26   
331          swan river      CA  52.1058 -101.2676        40   
436        lakki marwat      PK  32.6079   70.9114        20   
441         yangliuqing      CN  39.1375  116.9997        31   

                               Hotel Name  
29                         No hotel found  
73                         No hotel found  
96                     Zhangye Hotel 张掖饭店  
141                        Ant Street Inn  
174                        No hotel found  
176                    Residencial Marian  
231  Days Inn North Dallas/Farmers Branch  
239                       Hotel Deportivo  
243                        No hotel found  
258                        No hotel found  
268                              紫御国际假日酒店  
331                    Swan River Super 8  
436                        No hotel found  
441                                IU Inn

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

hotel_map_plot2 = hotel_df.hvplot.points(
                                x="Lng",y= "Lat",
                                title='Hotel Locations',
                                geo = True,
                                tiles = "OSM",
                                frame_width = 800,
                                frame_height = 550, 
                                size = "Humidity",
                                scale = 1,
                                color = "City",
                                hover_cols=['Hotel Name', 'City', 'Country']
                            )

# Display the map 
hotel_map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)